In [ ]:
import numpy as np
import pandas as pd
from typing import List, Tuple, Union
import matplotlib.pyplot as plt
import seaborn as sns
from modules import utils, model, metrics
import itertools


In [ ]:
# Default plot settings
plt.rcParams["figure.dpi"] = 300
sns.set()


In [ ]:
pd.core.common.random_state(None)
seed = 105
np.random.seed(seed)

In [ ]:
# def generate_data(n_rows:int=1000):
#     # column_patterns = [np.random.uniform(1, 72, n_rows),
#     #                    np.random.uniform(18, 118, n_rows),
#     #                    np.random.uniform(100, 1000, n_rows),
                       
#     #                    ]
#     data_list = [
#         np.random.uniform(1, 72, n_rows),
#         np.random.uniform(18, 118, n_rows),
#         np.random.uniform(100, 1000, n_rows),
#         np.random.choice([0,1], size = n_rows, p = [0.25, 0.75])
#                 ]
#     data_dict = {}
#     for i, d in enumerate(data_list):
#         data_dict[f"x{i}"] = d
#     data = pd.DataFrame(data_dict)

#     # print(training_data.shape, npdata.shape)
#     # print(training_data)
    
#     return data

In [ ]:
url = "https://raw.githubusercontent.com/VincentGranville/Main/main/Telecom.csv"
data = pd.read_csv(url)

features = ['tenure', 'MonthlyCharges', 'TotalCharges','Churn']
data = data[features]
# for col in data.select_dtypes(include=[object]).columns:
#     data[col] = utils.label_encode(data[col])



In [ ]:
data.info()

In [ ]:
#data = generate_data(6000)

url = "https://raw.githubusercontent.com/VincentGranville/Main/main/Telecom.csv"
data = pd.read_csv(url)

features = ['tenure', 'MonthlyCharges', 'TotalCharges','Churn']
data = data[features]

# data['Churn'] = data['Churn'].map(category_to_integer)
data['TotalCharges'].replace(' ', np.nan, inplace=True)
data.dropna(subset=['TotalCharges'], inplace=True) # remove missing data
arr1 = data['tenure'].to_numpy()
arr2 = data['TotalCharges'].to_numpy()
arr2 = arr2.astype(float)
residues = arr2 - arr1 * np.sum(arr2) / np.sum(arr1) # also try arr2/arr1
data['TotalChargeResidues'] = residues

print(data.head())
print (data.shape)
print (data.columns)


In [ ]:
features = ['tenure', 'MonthlyCharges', 'TotalChargeResidues','Churn']
data = data[features]

for col in data.select_dtypes(include=[object]).columns:
    data[col] = utils.label_encode(data[col])

print(data.head())
# Split data into Train & Validation sets
training_data  = data.sample(frac = 0.5)
validation_data = data.drop(training_data.index)
# np_tr = np.array(training_data)
# np_val = np.array(validation_data)

print(f"Data Shape: {data.shape}\nTraining Shape: {training_data.shape}\nValidation Shape: {validation_data.shape}")

In [ ]:
# # Get Student's Success Data
# data = utils.get_cleaned_students_data()

# # Split data into Train & Validation sets
# training_data  = data.sample(frac = 0.5)
# validation_data = data.drop(training_data.index)

# print(f"Data Shape: {data.shape}\nTraining Shape: {training_data.shape}\nValidation Shape: {validation_data.shape}")

In [ ]:
# def scott_bin_size(data):
#     n = len(data)
#     h = 3.5 * np.std(data) / (n**(1/3))
#     k = int(np.ceil((np.max(data) - np.min(data)) / h))
#     return k

# def freedman_diaconis_bin_size(data):
#     n = len(data)
#     iqr = np.percentile(data, 75) - np.percentile(data, 25)
#     h = 2 * iqr / (n**(1/3))
#     k = int(np.ceil((np.max(data) - np.min(data)) / h))
#     return k

In [ ]:
bins = [50, 40, 40, 4]
#features = training_data.columns
n_synth_rows = len(validation_data)
nogan = model.NoGANSynth(data = training_data,
                   bins = bins
                   )
nogan.create_bin_keys()
synth_data = nogan.generate_synthetic_data(n_synth_rows)
synth_data['Churn'] = synth_data['Churn'].astype('int') 
synth_data['tenure'] = synth_data['tenure'].astype('int')
print(synth_data)

In [ ]:
# bin_list = []

# for skey in nogan.bin_keys:
#     bin_list.append((skey, nogan.bin_keys[skey]["frequency"]))
    
# bin_list.sort(key=lambda item: item[1], reverse=True)
# bin_list_df = pd.DataFrame(bin_list, columns = ["key","freq"])
# bin_list_df[:5]

In [ ]:
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt

# # Create a sample pandas DataFrame (replace this with your dataset)
# data = {
#     'variable1': np.random.randn(100),
#     'variable2': np.random.randn(100),
#     'variable3': np.random.randn(100)
# }
# df = pd.DataFrame(data)

# # Function to calculate ECDF for a given dataset
# def ecdf(data):
#     n = len(data)
#     x = np.sort(data)
#     y = np.arange(1, n+1) / n
#     return x, y

# # Create a new figure
# plt.figure(figsize=(10, 6))

# # Plot ECDF for each variable
# for col in df.columns:
#     x, y = ecdf(df[col])
#     plt.plot(x, y, marker='.', linestyle='none', label=col)

# # Add labels and legend
# plt.xlabel('Value')
# plt.ylabel('ECDF')
# plt.title('Multivariate ECDF')
# plt.legend()

# # Show the plot
# plt.show()


In [ ]:
ecdf_validation1 = metrics.compute_ecdf(validation_data, 1000, False)
print("")
ecdf_validation2 = metrics.compute_ecdf(validation_data, 1000, True)

In [ ]:
ks_max1, ecdf_real1, ecdf_synth1 = metrics.ks_delta(synth_data, ecdf_validation1)
ks_max2, ecdf_real2, ecdf_synth2 = metrics.ks_delta(synth_data, ecdf_validation2)
ks_max = max(ks_max1, ks_max2)
print(f"ECDF Kolmogorof-Smirnov dist. (synth. vs valid.): {ks_max:6.4f}")

base_ks_max1, ecdf_val1, ecdf_train1 = metrics.ks_delta(training_data, ecdf_validation1)
base_ks_max2, ecdf_val2, ecdf_train2 = metrics.ks_delta(training_data, ecdf_validation2)
base_ks_max = max(base_ks_max1, base_ks_max2)
print(f"Base ECDF Kolmogorof-Smirnov dist. (train. vs valid.): {base_ks_max:6.4f}")

In [ ]:
# import numpy as np
# import pandas as pd
# from scipy.stats import ks_2samp

# # Create two sample pandas DataFrames (replace with your data)
# np.random.seed(42)
# data1 = pd.DataFrame(np.random.randn(100, 4), columns=['feature1', 'feature2', 'feature3', 'feature4'])
# data2 = pd.DataFrame(np.random.randn(100, 4), columns=['feature1', 'feature2', 'feature3', 'feature4'])

# # Define the number of permutations for resampling
# num_permutations = 1000

# # Initialize an array to store permutation KS statistics for each feature
# permuted_ks_stats = {col: [] for col in data1.columns}

# # Perform permutation testing
# for _ in range(num_permutations):
#     combined_data = pd.concat([data1, data2])
#     combined_data_shuffled = combined_data.sample(frac=1, replace=False, random_state=None)
    
#     shuffled_data1 = combined_data_shuffled.iloc[:len(data1)]
#     shuffled_data2 = combined_data_shuffled.iloc[len(data1):]
    
#     for col in data1.columns:
#         permuted_ks_statistic, _ = ks_2samp(shuffled_data1[col], shuffled_data2[col])
#         permuted_ks_stats[col].append(permuted_ks_statistic)

# # Calculate the observed KS statistics for each feature
# observed_ks_stats = {col: ks_2samp(data1[col], data2[col])[0] for col in data1.columns}

# # Calculate p-values for each feature based on the permutation results
# p_values = {col: (np.sum(np.array(permuted_ks_stats[col]) >= observed_ks_stats[col]) + 1) / (num_permutations + 1) for col in data1.columns}

# # Define the significance level
# alpha = 0.05

# # Check if any of the p-values are less than the significance level to reject the null hypothesis
# features_different = [col for col in p_values if p_values[col] < alpha]

# if features_different:
#     print(f"Null hypothesis rejected for features: {', '.join(features_different)}.")
# else:
#     print("Null hypothesis not rejected. Distributions are not significantly different for any features.")


In [ ]:
features = ["tenure","MonthlyCharges", "TotalChargeResidues", "Churn"]
def plot_histogram(dfs, dfv, features, title):
    num_graphs = len(features) * 2
    num_cols = 2
    num_rows = num_graphs // num_cols
    # Create subplots
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 4 * num_rows))
    axes = axes.flatten()
    for i, ax in enumerate(axes):
        column_name = features[i//2]
        if i%2 == 0:
            sns.histplot(data = dfv, x = column_name, ax = ax)
        else:
            sns.histplot(data = dfs, x = column_name, ax = ax)
    plt.suptitle(title)

def plot_scatter(dfs, dfv, features, title):
    feature_combinations = list(itertools.combinations(features, 2))
    #print(feature_combinations)
    num_graphs = len(feature_combinations) * 2
    num_cols = 2
    num_rows = num_graphs // num_cols
    #feature_idx = 0
    # Create subplots
    fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, 4 * num_rows))
    axes = axes.flatten()
    for i, ax in enumerate(axes):
        x_col = feature_combinations[i // 2][0]
        y_col = feature_combinations[i // 2][1]
        if i%2 == 0:
            sns.scatterplot(data = dfv, x = x_col, y = y_col, ax = ax)
        else:
            sns.scatterplot(data = dfs, x = x_col, y = y_col, ax = ax)
        
    plt.suptitle(title)

   

In [ ]:
plot_histogram(synth_data, validation_data, features,
               title = "Histogram Comparisons (Synthetic vs Validation)") 

plot_scatter(synth_data, validation_data, features,
             title = "Scatter Plot Comparisons (Synthetic vs Validation) ")    